## Import Libraries and Data

In [155]:
# libraries to use
from pathlib import Path
import plotly.express as px
import pandas as pd
import hvplot.pandas
import holoviews as hv
import plotly.graph_objects as go
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.covariance import EmpiricalCovariance
from sklearn.metrics import silhouette_score

from keras.wrappers.scikit_learn import KerasClassifier

from scipy.spatial import distance

import tensorflow as tf

import itertools

import numpy as np

In [161]:
# Save CSV's as pandas DF variables
tool_0028AA_df = pd.read_csv("Resources/Output_data/tool_0028AA_df.csv")
tool_9622C_df = pd.read_csv("Resources/Output_data/tool_9622C_df.csv")

In [9]:
# Step 1 Done
# Merge two tools together to get more geophysical to use, return merged df and a list of prefixes to be used in clustering/classification of outliers
def MergeData(df1, df2):
    merged_df = pd.merge(df1, df2, on=['well', 'Depth_M'], how='inner')
    prefix_list = merged_df['well'].str.extract('^(.*?)\d+')[0].unique()
    prefix_list = prefix_list.astype(str).tolist()
    prefix_list = [prefix for prefix in prefix_list if prefix and prefix != 'nan']
    return merged_df, prefix_list


['RHGC', 'CBD', 'CCD', 'RHRC']


In [122]:
# Step 2 Done
def CleanFilterData(df, prefix_list):
    output_dataframes = {}
    
    for prefix in prefix_list:
        filtered_df = df[df['well'].str.startswith(prefix)]
        merged_cleaned_df = filtered_df[['well', 'Depth_M', 'SUSCEP_CGS E-5', 'DENSITY_G/CC', 'GAM(NAT)_CPS']]
        merged_data = merged_cleaned_df.replace(-999.25, pd.NA)
        merged_data.dropna(inplace=True)
        merged_filtered_data = merged_data[
            (merged_data["SUSCEP_CGS E-5"] >= 0) &
            (merged_data["DENSITY_G/CC"] >= 0)&
            (merged_data["SUSCEP_CGS E-5"] <= 200) &
            (merged_data['GAM(NAT)_CPS'] <= 100)
        ].copy()
        #merged_filtered_data = merged_filtered_data.drop(columns=['well', 'Depth_M'])
        
        output_dataframes[prefix] = merged_filtered_data
    
    return output_dataframes

In [145]:
def performClustering(output_data, prefix_list):
    output_results = {}
    columns_to_scale = ['SUSCEP_CGS E-5', 'DENSITY_G/CC', 'GAM(NAT)_CPS']

    for prefix in prefix_list:
        # Get the filtered DataFrame for the prefix
        merged_filtered_data = output_data[prefix]

        # Perform silhouette score to determine the number of clusters
        silhouette_scores = []
        cluster_range = range(2, 11)

        for num_clusters in cluster_range:
            kmeans = KMeans(n_clusters=num_clusters)
            kmeans.fit(merged_filtered_data[columns_to_scale])  # Use only the specified columns
            labels = kmeans.labels_
            silhouette_scores.append(silhouette_score(merged_filtered_data[columns_to_scale], labels))  # Use only the specified columns

        optimal_num_clusters = cluster_range[silhouette_scores.index(max(silhouette_scores))]

        if optimal_num_clusters == 1:
            output_results[prefix] = 1
        else:
            # Perform clustering and outlier detection using the determined number of clusters
            scaler = StandardScaler()
            scaled_data = scaler.fit_transform(merged_filtered_data[columns_to_scale])

            k = optimal_num_clusters
            mahalanobis_distances = np.empty((len(merged_filtered_data),))

            model = KMeans(n_clusters=k)
            model.fit(scaled_data)
            df_clustered = pd.DataFrame(merged_filtered_data[columns_to_scale], columns=columns_to_scale)
            df_clustered['cluster_label'] = model.labels_

            for cluster_label in np.unique(df_clustered['cluster_label']):
                cluster_data = df_clustered.loc[df_clustered['cluster_label'] == cluster_label, ["DENSITY_G/CC", "SUSCEP_CGS E-5", 'GAM(NAT)_CPS']]
                if len(cluster_data) > 1:
                    envelope = EllipticEnvelope()
                    envelope.fit(cluster_data)
                    cluster_distances = envelope.mahalanobis(cluster_data)
                    mahalanobis_distances[df_clustered['cluster_label'] == cluster_label] = cluster_distances

            df_clustered['Mahalanobis_Distance'] = mahalanobis_distances
            threshold = 20
            df_clustered['is_outlier'] = df_clustered['Mahalanobis_Distance'] > threshold

            output_results[prefix] = df_clustered

    return output_results, optimal_num_clusters

In [150]:
#working
merged_df, prefix_list = MergeData(tool_0028AA_df, tool_9622C_df)
undersampled_data = merged_df.sample(n=20000)
#working
output_data = CleanFilterData(undersampled_data, prefix_list)

output_results, clusters = performClustering(output_data, prefix_list)
print(clusters)
output_results

3


{'RHRC':          SUSCEP_CGS E-5  DENSITY_G/CC  GAM(NAT)_CPS  cluster_label  \
 100073         3.934661      2.156725     36.666668              1   
 1531611        5.565672      2.906614     71.250000              1   
 166437        52.809322      2.841934     54.285713              1   
 1467583        5.154344      1.917626     54.285713              1   
 1424356        6.966211      2.412937     76.250000              1   
 ...                 ...           ...           ...            ...   
 1204213       13.758514      2.196316     61.428570              1   
 641344       102.571228      2.916018     91.111115              2   
 979059         9.859686      3.251400     65.000000              0   
 952173        19.310434      2.796513     22.222221              0   
 700699        10.741177      1.682290     65.714287              1   
 
          Mahalanobis_Distance  is_outlier  
 100073               3.129420       False  
 1531611              3.852154       False  
 16

In [151]:
df = pd.concat({key: pd.DataFrame(value) for key, value in output_results.items()}, axis=1)
df.reset_index(drop=True, inplace=True)
df

RHRC                                          \
      SUSCEP_CGS E-5 DENSITY_G/CC GAM(NAT)_CPS cluster_label   
0           3.934661     2.156725    36.666668             1   
1           5.565672     2.906614    71.250000             1   
2          52.809322     2.841934    54.285713             1   
3           5.154344     1.917626    54.285713             1   
4           6.966211     2.412937    76.250000             1   
...              ...          ...          ...           ...   
15300      13.758514     2.196316    61.428570             1   
15301     102.571228     2.916018    91.111115             2   
15302       9.859686     3.251400    65.000000             0   
15303      19.310434     2.796513    22.222221             0   
15304      10.741177     1.682290    65.714287             1   

                                       
      Mahalanobis_Distance is_outlier  
0                 3.129420      False  
1                 3.852154      False  
2                84.753697       True  
3                 1.420662      False  
4                 0.963449      False  
...                    ...        ...  
15300             0.925434      False  
15301             2.207815      False  
15302             6.581389      False  
15303             9.956289      False  
15304             3.222715      False  

[15305 rows x 6 columns]

In [159]:
scatter_plot = df['RHRC'].hvplot.scatter(
    x='DENSITY_G/CC',
    y='SUSCEP_CGS E-5',
    color='is_outlier',
)

hvplot.show(scatter_plot)

Launching server at http://localhost:65037


In [160]:
fig = go.Figure(data=go.Scatter3d(
    x=df['RHRC']['SUSCEP_CGS E-5'],
    y=df['RHRC']['DENSITY_G/CC'],
    z=df['RHRC']['GAM(NAT)_CPS'],
    mode='markers',
    marker=dict(
        size=2,  # Adjust the size here (e.g., 2 for half the size)
        color=df['RHRC']['cluster_label'],
        colorscale='Viridis',
        opacity=0.8
    )
))

fig.show()

In [ ]:
# can be removed if not needed
def clusterEmpiricalCovariance(df, n_clusters):
    scaled_merged_df = merged_filtered_data.drop(columns=['well', 'Depth_M'])
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(scaled_merged_df)

    k = n_clusters
    mahalanobis_distances = []

    model = KMeans(n_clusters=k)
    model.fit(scaled_data)
    scaled_merged_df = pd.DataFrame(scaled_data, columns=['SUSCEP_CGS E-5', 'DENSITY_G/CC'])
    scaled_merged_df['cluster_label'] = model.labels_

    for i in range(k):
        cluster_points = scaled_merged_df.loc[scaled_merged_df['cluster_label'] == i, ['SUSCEP_CGS E-5', 'DENSITY_G/CC']]
        cov = EmpiricalCovariance().fit(cluster_points)
        cluster_center_reshaped = np.reshape(model.cluster_centers_[i], (1, -1))
        mahalanobis_dist = distance.cdist(cluster_points, cluster_center_reshaped, 'mahalanobis', VI=cov.covariance_)
        mahalanobis_distances.extend(mahalanobis_dist)

    scaled_merged_df['min_mahalanobis_distance'] = np.min(mahalanobis_distances, axis=1)

    threshold = np.mean(scaled_merged_df['min_mahalanobis_distance']) + 3 * np.std(scaled_merged_df['min_mahalanobis_distance'])
    scaled_merged_df['is_outlier'] = scaled_merged_df['min_mahalanobis_distance'] > threshold

    return EmpiricalCovariance_df



In [18]:
merged_df, prefix_list = MergeData(tool_0028AA_df, tool_9622C_df)
output_dataframes = CleanFilterData(merged_df, prefix_list)
output_dataframes

{'RHRC':          SUSCEP_CGS E-5  DENSITY_G/CC
 13            13.890224      2.253303
 14            14.381913      2.131638
 15            18.192505      2.212196
 16            21.265564      2.315750
 17            18.772545      2.231368
 ...                 ...           ...
 2278653        4.346225      1.979177
 2278654        5.463825      1.893722
 2278655        5.494870      1.961375
 2278656        5.774270      2.062066
 2278657        6.177848      1.646185
 
 [2173879 rows x 2 columns]}

In [ ]:
tool_0028AA_df.head()

In [ ]:
tool_9622C_df.describe()

In [ ]:
tool_9622C_df.head()
# 'SUSCEP_CGS E-5' 'SANGB_DEG' 'TEMP_CPS' 

In [ ]:
# join the datasets together based on well and Depth_M
merged_df = pd.merge(tool_0028AA_df, tool_9622C_df, on=['well', 'Depth_M'], how='inner')

In [ ]:
merged_df.columns

## Clean and Prepare Data

In [ ]:
#REMOVE LATER
merged_df = merged_df.sample(n=500)

In [ ]:
# Filter for a single well prefix RH
prefix_counts = merged_df['well'].str.extract('^(.*?)\d+')[0].value_counts()
prefix_counts.head(10)

In [ ]:
# prefix_counts = tool_0028A_df['well'].str.extract('^(.*?)\d+')[0].value_counts()
# prefix_counts.head(10)

In [ ]:
filtered_df = merged_df[merged_df['well'].str.startswith('RHRC')]
filtered_df.tail()

In [ ]:
# Remove unnecessary columns
merged_cleaned_df = filtered_df[['well', 'Depth_M', 'SUSCEP_CGS E-5', 'DENSITY_G/CC']]

In [ ]:
# Clean nulls
merged_data = merged_cleaned_df.replace(-999.25, pd.NA)
merged_data.dropna(inplace=True)
merged_data.describe()

In [ ]:
# Prior to scaling, obviously invalid data such as negative Susceptability readings and datapoints that fall well outside the tools expected rages will be removed
# filter out rows where Susceptability falls below the tools specified minimum value using tool documentation https://www.century-geo.com/9622
merged_filtered_data = merged_data[
    (merged_data["SUSCEP_CGS E-5"] >= 0) &
    (merged_data["DENSITY_G/CC"] >= 0) #&
    #(merged_data["DENSITY_G/CC"] <= 6) &
    #(merged_data["SUSCEP_CGS E-5"] <= 35)
].copy()

merged_filtered_data.describe()

In [ ]:
# remove rows that contain outliers that fall outside 3 standard deviations for the columns in order to create clusters on legitimate datapoints
# Calculate the mean and standard deviation for each column
mean_values = merged_filtered_data.mean()
std_values = merged_filtered_data.std()

# Define the upper and lower bounds for filtering
lower_bounds = mean_values - (3 * std_values)
upper_bounds = mean_values + (3 * std_values)
merged_filtered_data = merged_filtered_data[
    (merged_filtered_data['SUSCEP_CGS E-5'] >= lower_bounds['SUSCEP_CGS E-5']) &
    (merged_filtered_data['SUSCEP_CGS E-5'] <= upper_bounds['SUSCEP_CGS E-5']) &
    (merged_filtered_data['DENSITY_G/CC'] >= lower_bounds['DENSITY_G/CC']) &
    (merged_filtered_data['DENSITY_G/CC'] <= upper_bounds['DENSITY_G/CC']) #&
    #(merged_filtered_data['TEMP_CPS'] >= lower_bounds['TEMP_CPS']) &
    #(merged_filtered_data['TEMP_CPS'] <= upper_bounds['TEMP_CPS'])
]

merged_filtered_data.describe()

# Predict clusters using Empirical Covariance

In [ ]:
# Scale the filtered Data Remove data that is not to be scaled (Well and Depth)
scaled_merged_filtered_data = merged_filtered_data.drop(columns=['well', 'Depth_M'])
scaler = StandardScaler()
scaled_data = scaler.fit_transform(scaled_merged_filtered_data)

In [ ]:
# Choose number of clusters
k = 4
mahalanobis_distances = []

model = KMeans(n_clusters=k)
model.fit(scaled_data)
scaled_merged_df = pd.DataFrame(scaled_data, columns=['SUSCEP_CGS E-5', 'DENSITY_G/CC'])
scaled_merged_df['cluster_label'] = model.labels_

for i in range(k):
    cluster_points = scaled_merged_df.loc[scaled_merged_df['cluster_label'] == i, ['SUSCEP_CGS E-5', 'DENSITY_G/CC']]
    cov = EmpiricalCovariance().fit(cluster_points)
    cluster_center_reshaped = np.reshape(model.cluster_centers_[i], (1, -1))
    mahalanobis_dist = distance.cdist(cluster_points, cluster_center_reshaped, 'mahalanobis', VI=cov.covariance_)
    mahalanobis_distances.extend(mahalanobis_dist)

In [ ]:
scaled_merged_df['min_mahalanobis_distance'] = np.min(mahalanobis_distances, axis=1)

In [ ]:
# Determine outliers based on a threshold (e.g., 3 standard deviations from the mean) CAN BE CHANGED DEPENDING ON OUTPUT
threshold = np.mean(scaled_merged_df['min_mahalanobis_distance']) + 3 * np.std(scaled_merged_df['min_mahalanobis_distance'])
scaled_merged_df['is_outlier'] = scaled_merged_df['min_mahalanobis_distance'] > threshold

In [ ]:
scaled_merged_df

In [ ]:
# Plot the DataFrame
scaled_merged_df.hvplot.scatter(
    x="SUSCEP_CGS E-5",
    y="DENSITY_G/CC",
    color="cluster_label"
)

In [ ]:
# Plot the DataFrame
scaled_merged_df.hvplot.scatter(
    x="SUSCEP_CGS E-5",
    y="DENSITY_G/CC",
    color="is_outlier"
)

# Predict outliers using Elliptic Enveliope

In [ ]:
# Choose number of clusters
k = 3
mahalanobis_distances = np.empty((len(scaled_merged_df),))  # Initialize an empty NumPy array

model = KMeans(n_clusters=k)
model.fit(scaled_data)
scaled_merged_df = pd.DataFrame(scaled_data, columns=['SUSCEP_CGS E-5', 'DENSITY_G/CC'])
scaled_merged_df['cluster_label'] = model.labels_

In [ ]:
for cluster_label in np.unique(scaled_merged_df['cluster_label']):
    cluster_data = scaled_merged_df.loc[scaled_merged_df['cluster_label'] == cluster_label, ["DENSITY_G/CC", "SUSCEP_CGS E-5"]]  # Adjust the features accordingly

    # Fit the Elliptic Envelope on the cluster data
    envelope = EllipticEnvelope()
    envelope.fit(cluster_data)

    # Calculate the Mahalanobis distance for each data point in the cluster
    cluster_distances = envelope.mahalanobis(cluster_data)

    # Assign the Mahalanobis distances to the corresponding indices in the mahalanobis_distances array
    mahalanobis_distances[scaled_merged_df['cluster_label'] == cluster_label] = cluster_distances

In [ ]:
# Calculate the average Mahalanobis distance across all clusters
scaled_merged_df['Mahalanobis_Distance'] = mahalanobis_distances

In [ ]:
# Set a threshold to determine outliers
threshold = 8  # Adjust as needed

# Identify outliers based on the threshold
scaled_merged_df['is_outlier'] = scaled_merged_df['Mahalanobis_Distance'] > threshold
scaled_merged_df

In [ ]:
# Plot outliers with data to see if it is classifying properly
scaled_merged_df.hvplot.scatter(
    x="SUSCEP_CGS E-5", 
    y="DENSITY_G/CC", 
    by="cluster_label"
)

In [ ]:
# Plot outliers with data to see if it is classifying properly
scaled_merged_df.hvplot.scatter(
    x="SUSCEP_CGS E-5", 
    y="DENSITY_G/CC", 
    by="is_outlier"
)

## Model optimisation

In [ ]:
class CustomCSVLogger(CSVLogger):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 == 0:
            super().on_epoch_end(epoch, logs)

In [ ]:
# Tuning Parameters
activation_functions = ['relu', 'sigmoid', 'tanh']
hidden_nodes_layer1_values = [32, 64, 128]
hidden_nodes_layer2_values = [16, 32, 64]
optimizers = ['adam', 'rmsprop']
losses = ['binary_crossentropy', 'mean_squared_error']

In [ ]:
parameter_combinations = list(itertools.product(activation_functions, hidden_nodes_layer1_values, hidden_nodes_layer2_values, optimizers, losses))
parameter_combinations[103]

In [ ]:
def train_model(activation, hidden_nodes_layer1, hidden_nodes_layer2, optimizer, loss):
    nn = tf.keras.models.Sequential()
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation))
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=activation))
    nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    nn.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    nn.fit(X_train_scaled, y_train, epochs=20, callbacks=[CustomCSVLogger('model_tuning_results.csv', append=True)])

In [ ]:
results_df = pd.DataFrame(columns=['Activation', 'Hidden Nodes Layer 1', 'Hidden Nodes Layer 2', 'Optimizer', 'Loss'])
for params in parameter_combinations:
    activation, hidden_nodes_layer1, hidden_nodes_layer2, optimizer, loss = params
    train_model(activation, hidden_nodes_layer1, hidden_nodes_layer2, optimizer, loss)
    results_df = results_df.append({'Activation': activation,
                                    'Hidden Nodes Layer 1': hidden_nodes_layer1,
                                    'Hidden Nodes Layer 2': hidden_nodes_layer2,
                                    'Optimizer': optimizer,
                                    'Loss': loss}, ignore_index=True)
    
results_df.to_csv('model_parameters.csv', index=False)